### Utilities are defined in the cell below. 
IF YOU ARE NEW THEN <br/>
#1. INSTALL VSCODE FIRST (avoid google colab because webdriver needs too many packages to install to run a browser in google colab)<br/>
#2. GO TO EXTENSIONS, install python and jupyter notebook <br/>
#3.  RUN 'pip install selenium' in vs code terminal (ctrl +shift+ local button ) <br/>
#4. RUN 'pip install pandas' <br/>
#5. Run any other package if you machine does not have it. Usually you will see an error which says 'can't identify pandas or selenium' <br/>


In [1]:

from selenium import webdriver #Webdriver in selenium is a framework which can be used to automated acess to go to cross-platform. 
#In Selenium package the framework webdriver needs few functions to run the automation smoothly. 
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as ec
from selenium.webdriver.support.select import Select
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import TimeoutException

##For FIREFOX AS BROWSER 
#from selenium.webdriver.firefox.service import Service 

##FOR CHROME AS BROWSER 
from selenium.webdriver.chrome.service import Service
from chromedriver_py import binary_path              # this will get you the path variable

##OTHER THAN SELENIUM, WE NEED FEW PYTHON PACKAGES TO GET THE DATASET SAVE, HANDLE IN THE CODE WELL 

import time
import pandas as pd
import random 
import csv 
import math
from pathlib import Path

### FROM WEBDRIVER FRAMEWORK WHICH CROSS-PLATFORM CLASS SHOULD BE USE. 
#Here I am using chrome. FIreforx can also be used. <br/>
#option = webdriver.FirefoxOptions() <br/>
#add argument is the way, you can add what should the website incorporate. For example, I can only allow for text elements of website to show or no cookies or no images. <br/>
#in Firefox, we use set_preference function. <br/>

In [2]:
option = webdriver.ChromeOptions()
option.add_argument('log-level=3')
option.add_argument('--ignore-certificate-errors-spki-list')
option.add_argument('--ignore-ssl-errors')
# options.add_argument('--headless')
option.add_argument('--disable-gpu')
#service_object = Service(binary_path)



### How to access the url/webpage 
#1. Just start with webdriver and choose a browser, add the options for the webpage

In [3]:

##when webdriver will open chrome, it's gonna add the options as we have defined above and se
driver = webdriver.Chrome(service=Service(), options=option) 

##The below can be used to access as well. 

#driver= webdriver.Chrome(options=option)
#maximize the crhoem window 
driver.maximize_window()
from selenium.common.exceptions import TimeoutException

t = time.time()

##sometimes browser can take time to appear, to set the timeout; we have an upper limit. You can change that accordingly. 
driver.set_page_load_timeout(500)

# webpage I want to access is the one below 
url = 'https://dmf.gujarat.gov.in/'

try:
    #url 
    driver.get(url)
except TimeoutException:
    #if website can't be opened in 500 seconds then the window will stop trying after that and will print the statement below. 
    driver.execute_script("window.stop();")
print('Time consuming:', time.time() - t)

assert "DMF" in driver.title
time.sleep(1)


#In case if there is a pop-up 
try: 
     WebDriverWait(driver, 10).until(ec.element_to_be_clickable((By.CLASS_NAME, 'close'))).click()

except: 
    WebDriverWait(driver, 5).until(ec.alert_is_present())
    driver.switch_to.alert.accept()




Time consuming: 341.78294825553894


In [4]:
###Once the website is opened in the chrome then I wanna hover on district element on the page. 
# XPATH can be copied -- bring your cursor/mouse to the element whose path you want then type 'ctrl+shift+I' then right click, go to copy element---copy xpath will be there 

a= ActionChains(driver)
element = driver.find_element(By.XPATH, '/html/body/div[1]/header/div/div[1]/div[2]/div/div/div/div[3]/div/nav/ul/li[6]/a')
hover = a.move_to_element(element).perform()

#Calculate the district_urls - from A & B categories 
temp_urls1 = driver.find_elements(By.XPATH, '/html/body/div[1]/header/div/div[1]/div[2]/div/div/div/div[3]/div/nav/ul/li[6]/div/ul[//a[position()>0 and position()<3]]/li[//a[position()>0]]/a')
dist_urls1 =[i.get_attribute("href") for i in temp_urls1]

#For C-Category Districts 
temp_urls2 = driver.find_elements(By.XPATH, '/html/body/div[1]/header/div/div[1]/div[2]/div/div/div/div[3]/div/nav/ul/li[6]/div/ul[3]/li[2]/ul[//a[position()>1]]/li[//a[position()>0]]/a')
dist_urls2 =[i.get_attribute("href") for i in temp_urls2]

#Make the final list of districts and district_urls 
district_urls = dist_urls1+dist_urls2

print(district_urls) 


['https://dmf.gujarat.gov.in/District/Index/Amreli', 'https://dmf.gujarat.gov.in/District/Index/Devbhoomidwarka', 'https://dmf.gujarat.gov.in/District/Index/Girsomnath', 'https://dmf.gujarat.gov.in/District/Index/Kachchh', 'https://dmf.gujarat.gov.in/District/Index/Porbandar', 'https://dmf.gujarat.gov.in/District/Index/Aravalli', 'https://dmf.gujarat.gov.in/District/Index/Banaskantha', 'https://dmf.gujarat.gov.in/District/Index/Bharuch', 'https://dmf.gujarat.gov.in/District/Index/Bhavnagar', 'https://dmf.gujarat.gov.in/District/Index/Jamnagar', 'https://dmf.gujarat.gov.in/District/Index/Junagadh', 'https://dmf.gujarat.gov.in/District/Index/Navsari', 'https://dmf.gujarat.gov.in/District/Index/Sabarkantha', 'https://dmf.gujarat.gov.in/District/Index/Surat', 'https://dmf.gujarat.gov.in/District/Index/Surendranagar', 'https://dmf.gujarat.gov.in/District/Index/Vadodara', 'https://dmf.gujarat.gov.in/District/Index/Ahmedabad', 'https://dmf.gujarat.gov.in/District/Index/Anand', 'https://dmf.gu

In [5]:
###collection of project data 
def project_details(url): 

    # Switch to the new window and open new URL 
    driver.get(url)
    time.sleep(10) 
    ##create a set of lists of output to create a local dataframe 
    t_District= driver.find_element(By.XPATH, '/html/body/div[1]/section[1]/div/div/div/div/div[1]/div[2]/div[1]/div[1]/div/div[2]').text
    t_Block= driver.find_element(By.XPATH, '/html/body/div[1]/section[1]/div/div/div/div/div[1]/div[2]/div[1]/div[2]/div/div[2]').text
    t_Village = driver.find_element(By.XPATH, '/html/body/div[1]/section[1]/div/div/div/div/div[1]/div[2]/div[2]/div[1]/div/div[2]').text
    t_GP  =driver.find_element(By.XPATH, '/html/body/div[1]/section[1]/div/div/div/div/div[1]/div[2]/div[2]/div[2]/div/div[2]').text 
    t_Lat  = driver.find_element(By.XPATH, '/html/body/div[1]/section[1]/div/div/div/div/div[1]/div[2]/div[3]/div[1]/div/div[2]').text
    t_Long = driver.find_element(By.XPATH, '/html/body/div[1]/section[1]/div/div/div/div/div[1]/div[2]/div[3]/div[2]/div/div[2]').text
    t_Ptype = driver.find_element(By.XPATH, '/html/body/div[1]/section[1]/div/div/div/div/div[1]/div[2]/div[4]/div[1]/div/div[2]').text
    t_Sector = driver.find_element(By.XPATH, '/html/body/div[1]/section[1]/div/div/div/div/div[1]/div[2]/div[4]/div[2]/div/div[2]').text 
    t_Sub_Sector = driver.find_element(By.XPATH, '/html/body/div[1]/section[1]/div/div/div/div/div[1]/div[2]/div[5]/div[1]/div/div[2]').text
    t_Project_Title = driver.find_element(By.XPATH, '/html/body/div[1]/section[1]/div/div/div/div/div[1]/div[2]/div[5]/div[2]/div/div[2]').text
    t_Project_Status = driver.find_element(By.XPATH, '/html/body/div[1]/section[1]/div/div/div/div/div[1]/div[2]/div[6]/div[1]/div/div[2]').text 
    t_firm = driver.find_element(By.XPATH, '/html/body/div[1]/section[1]/div/div/div/div/div[1]/div[2]/div[6]/div[2]/div/div[2]').text
    t_PStart_Date = driver.find_element(By.XPATH, '/html/body/div[1]/section[1]/div/div/div/div/div[1]/div[2]/div[7]/div[1]/div/div[2]').text 
    t_PEnd_Date = driver.find_element(By.XPATH, '/html/body/div[1]/section[1]/div/div/div/div/div[1]/div[2]/div[7]/div[2]/div/div[2]').text
    t_Est_time = driver.find_element(By.XPATH, ' /html/body/div[1]/section[1]/div/div/div/div/div[1]/div[2]/div[8]/div[1]/div/div[2]').text
    t_Exe_Dept = driver.find_element(By.XPATH, '/html/body/div[1]/section[1]/div/div/div/div/div[1]/div[2]/div[8]/div[2]/div/div[2]').text
    t_Est_Amt =driver.find_element(By.XPATH, '/html/body/div[1]/section[1]/div/div/div/div/div[1]/div[2]/div[9]/div[1]/div/div[2]').text
    t_Appr_Amt = driver.find_element(By.XPATH, '/html/body/div[1]/section[1]/div/div/div/div/div[1]/div[2]/div[9]/div[2]/div/div[2]').text
    t_Exd_Amt = driver.find_element(By.XPATH, '/html/body/div[1]/section[1]/div/div/div/div/div[1]/div[2]/div[10]/div[1]/div/div[2]').text
    t_Exe_Mode= driver.find_element(By.XPATH, '/html/body/div[1]/section[1]/div/div/div/div/div[1]/div[2]/div[10]/div[2]/div/div[2]').text 
    t_Appr_Amt_By_EC = driver.find_element(By.XPATH, '/html/body/div[1]/section[1]/div/div/div/div/div[1]/div[2]/div[11]/div[1]/div/div[2]').text 
    t_Appr_Amt_By_GC = driver.find_element(By.XPATH, '/html/body/div[1]/section[1]/div/div/div/div/div[1]/div[2]/div[11]/div[2]/div/div[2]').text
    t_DOA = driver.find_element(By.XPATH, '/html/body/div[1]/section[1]/div/div/div/div/div[1]/div[2]/div[12]/div[1]/div/div[2]').text
    t_Des= driver.find_element(By.XPATH, '/html/body/div[1]/section[1]/div/div/div/div/div[1]/div[2]/div[14]/div/div/div[2]').text
    temp_list_out= [t_District, t_Block, t_Village, t_GP, t_Lat, t_Long, t_Ptype, t_Sector, t_Sub_Sector, t_Project_Title, t_Project_Status, t_firm, t_PStart_Date, t_PEnd_Date, t_Est_time, t_Appr_Amt,t_Exd_Amt, t_Exe_Dept, t_Est_Amt, t_Exe_Mode, t_Appr_Amt_By_EC, t_Appr_Amt_By_GC, t_DOA, t_Des ]
    return temp_list_out 

In [6]:
from io import StringIO
s = StringIO()

def print_stdout(var):
    print(var, file=s)
    result = s.getvalue()
    return print(result)



In [7]:
main_df = pd.DataFrame(index =[], columns = ['District', 'Block', 'Village', 'GP_Ward_Name', 'Latitude', 'Longitude', 'Priority_type', 'Sector_Name','Sub_Sector_Name', 'Project_Title','Project_Status','Contractor', 'Project_Start_Date', 'Project_End_Date', 'Estimation_Duration','Execution_Department','Estimated_Amouunt', 'Approved_Amount', 'Expenditure_Till_Now', 'Execution_Mode', 'Approved_Amount_By_EC', 'Approved_Amount_By_GC', 'Date_of_Agreement', 'Description' ]) 
master_list_out = []*24
u_District= []
u_Block= [] 
u_Village = []
u_GP  = []
u_Lat  =[]
u_Long = []
u_Ptype = []
u_Sector = []
u_Sub_Sector = []
u_Project_Title = []
u_Project_Status =[]
u_firm = []
u_PStart_Date = []
u_PEnd_Date = []
u_Est_time = []
u_Exe_Dept = []
u_Est_Amt =[]
u_Appr_Amt = []
u_Exd_Amt = []
u_Exe_Mode= [] 
u_Appr_Amt_By_EC = []
u_Appr_Amt_By_GC = []
u_DOA =[]
u_Des=[]


for i in district_urls: 
    temp_url = i 
    temp_url= temp_url.replace("Index","ProjectMap")
    #print(temp_url)
    driver.get(temp_url)
    time.sleep(5)
    #select block 
    Block_path = driver.find_elements(By.XPATH, '//*[@id="BlockId"]/option[position()>1]') 
    Block = [t.get_attribute("textContent") for t in Block_path]
    print(Block)
    len_b = int(len(Block)+1)
    for j in range(1, len_b, 1): 
        a = ActionChains(driver)
        element_blo = driver.find_element(By.ID, 'BlockId')
        a.move_to_element(element_blo).perform()
        Block_Select = Select(WebDriverWait(driver, 10).until(ec.element_to_be_clickable((By.ID, "BlockId"))))
        Block_Select.select_by_index(j)

        ##Select the Village 
        a.reset_actions()
        element_Vil1= driver.find_element(By.NAME, 'VillageCode')
        a.move_to_element(element_Vil1).perform()
        Village_path = driver.find_elements(By.XPATH, '//*[@id="VillageCode"]/option[position()>1]')
        Village = [p.get_attribute("textContent") for p in Village_path]
        print(Village)
        Village.remove('Other')
        print("Final_Village", Village)
        len_v =  int(len(Village)+1)
        for k in range(1, len_v, 1):
            a = ActionChains(driver)
            element_Vil2= driver.find_element(By.NAME, 'VillageCode')
            Village_select = Select(WebDriverWait(driver, 10).until(ec.element_to_be_clickable((By.ID, 'VillageCode'))))
            a.move_to_element(element_Vil2).perform()
            Village_select.select_by_index(k)
            #click on show button 
            driver.find_element(By.ID, "btnshow").click()
            time.sleep(10)
            ##get the maximum number of clicks 
            temp_var = driver.find_element(By.XPATH, '//*[@id="example_info"]')
            text_temp = temp_var.text.split(" ")
            max_clicks = math.floor(int(text_temp[-2])/10)+1
            max_var = list(range(1, max_clicks))
            project_url_path = driver.find_elements(By.XPATH, '//*[@id="example"]/tbody/tr[position()>0]/td[3]/a')
            project_url = [i.get_attribute("href") for i in project_url_path]
            for p in project_url:
                #Open a new window 
                driver.execute_script("window.open('');") 
                driver.switch_to.window(driver.window_handles[1]) 
                master_list_out = project_details(p)
                print(master_list_out)
                driver.close()
                driver.switch_to.window(driver.window_handles[0])
                u_District.append(master_list_out[0])
                u_Block.append(master_list_out[1])
                u_Village.append(master_list_out[2])
                u_GP.append(master_list_out[3])
                u_Lat.append(master_list_out[4])
                u_Long.append(master_list_out[5])
                u_Ptype.append(master_list_out[6])
                u_Sector.append(master_list_out[7])
                u_Sub_Sector.append(master_list_out[8])
                u_Project_Title.append(master_list_out[9])
                u_Project_Status.append(master_list_out[10])
                u_firm.append(master_list_out[11])
                u_PStart_Date.append(master_list_out[12])
                u_PEnd_Date.append(master_list_out[13])
                u_Est_time.append(master_list_out[14])
                u_Exe_Dept.append(master_list_out[15])
                u_Est_Amt.append(master_list_out[16])
                u_Appr_Amt.append(master_list_out[17])
                u_Exd_Amt.append(master_list_out[18])
                u_Exe_Mode.append(master_list_out[19])
                u_Appr_Amt_By_EC.append(master_list_out[20])
                u_Appr_Amt_By_GC.append(master_list_out[21])
                u_DOA.append(master_list_out[22])
                u_Des.append(master_list_out[23])
                temp_df = pd.DataFrame(list(zip(u_District, u_Block, u_Village, u_GP, u_Lat, u_Long, u_Ptype, u_Sector, u_Sub_Sector, u_Project_Title, u_Project_Status, u_firm, u_PStart_Date, u_PEnd_Date, u_Est_Amt, u_Exe_Dept, u_Est_Amt, u_Appr_Amt, u_Exd_Amt, u_Exe_Mode, u_Appr_Amt_By_EC, u_Appr_Amt_By_GC, u_DOA, u_Des)))
                row, col = temp_df.shape 
                print(temp_df.tail(5))                    
            for m in max_var: 
                for s in range(1,m,1): 
                    if s <max_var[-1] :       
                       driver.find_element(By.XPATH, '//*[@id="example_next"]/a').click() 
                    else: 
                       driver.back() 

            main_df = pd.concat([main_df, temp_df])
        

    
                
            




        




['Amreli', 'Babara', 'Bagasara', 'Dhari', 'Jafrabad', 'Khambha', 'Kukavav', 'Lathi', 'Liliya', 'Rajula', 'Savarkundla']
['Amreli City', 'Other']
Final_Village ['Amreli City']
[': AMRELI', ': Amreli', ': Amreli City', ': Amreli City', ': 21.609730', ': 71.225570', ':Other', ': Any Other Measure for Enhancing Environmental Quality in Mining District', ': Special Program', ': Chidrens play area (Game Equipments)', ': Completed', ':', ': 7/1/2019 1', ': 9/30/2019', ': 3', ': 300000.00', ': 300000.00', ': Municipal Office--Amreli', ': 335000.00', ': Contractor', ': 3/11/2017', ': 3/31/2017', ': 9/29/2017', ': Ward no.11 - Chidrens play area works in common plot of Krishnapark society (Game Equipments)']
         0         1              2              3            4            5   \
0  : AMRELI  : Amreli  : Amreli City  : Amreli City  : 21.609730  : 71.225570   

       6                                                  7   \
0  :Other  : Any Other Measure for Enhancing Environmenta...   



WebDriverException: Message: disconnected: not connected to DevTools
  (failed to check if window was closed: disconnected: not connected to DevTools)
  (Session info: chrome=119.0.6045.200)
Stacktrace:
	GetHandleVerifier [0x00007FF712A382B2+55298]
	(No symbol) [0x00007FF7129A5E02]
	(No symbol) [0x00007FF7128605AB]
	(No symbol) [0x00007FF71284E19B]
	(No symbol) [0x00007FF71284DDF0]
	(No symbol) [0x00007FF7128624A1]
	(No symbol) [0x00007FF7128DA18F]
	(No symbol) [0x00007FF7128C1E83]
	(No symbol) [0x00007FF71289670A]
	(No symbol) [0x00007FF712897964]
	GetHandleVerifier [0x00007FF712DB0AAB+3694587]
	GetHandleVerifier [0x00007FF712E0728E+4048862]
	GetHandleVerifier [0x00007FF712DFF173+4015811]
	GetHandleVerifier [0x00007FF712AD47D6+695590]
	(No symbol) [0x00007FF7129B0CE8]
	(No symbol) [0x00007FF7129ACF34]
	(No symbol) [0x00007FF7129AD062]
	(No symbol) [0x00007FF71299D3A3]
	BaseThreadInitThunk [0x00007FFCB791257D+29]
	RtlUserThreadStart [0x00007FFCB7AEAA58+40]


In [ ]:
max_t = [1, 2, 3, 4, 5]
for i in max_t: 
    print("i", i)
    for j in range(1, i, 1):
        print("j", j) 

1
1
2
1
2
3
1
2
3
4
